In [126]:
import pandas as pd
import urllib3
import re
import json
import numpy as np
import networkx as nx
from melonapi import scrapeMelon
import ast
import os
#import translators as ts
#ts.google

<h2>Clean the Kpop idols and groups dataframe<h2>

In [127]:
df_idols = pd.read_csv("./Dataset/kpop_idols.csv.xls")
df_boy_groups = pd.read_csv("./Dataset/kpop_idols_boy_groups.csv.xls")
df_girl_groups = pd.read_csv("./Dataset/kpop_idols_girl_groups.csv.xls")
df_idols

,Stage Name,Full Name,Korean Name,K. Stage Name,Date of Birth,Group,Country,Birthplace,Other Group,Gender
0,A.M,Seong Hyunwoo,성현우,에이엠,1996-12-31,Limitless,South Korea,NaN,NaN,M
1,Ace,Jang Wooyoung,장우영,에이스,1992-08-28,VAV,South Korea,NaN,NaN,M
2,Aeji,Kwon Aeji,권애지,애지,1999-10-25,Hashtag,South Korea,Daegu,NaN,F
3,AhIn,Lee Ahin,이아인,아인,1999-09-27,MOMOLAND,South Korea,Wonju,NaN,F
4,Ahra,Go Ahra,고아라,아라,2001-02-21,Favorite,South Korea,Yeosu,NaN,F
...,...,...,...,...,...,...,...,...,...,...
1305,Ziu,Park Heejun,박희준,지우,1997-06-16,VAV,South Korea,NaN,NaN,M
1306,ZN,Bae Jinye,배진예,지엔,1994-06-09,LABOUM,South Korea,Bucheon,UNI.T,F
1307,Zoa,Cho Hyewon,조혜원,조아,2005-05-31,Weeekly,South Korea,NaN,NaN,F
1308,Zuho,Bae Juho,백주호,주호,1996-07-04,SF9,South Korea,NaN,NaN,M


In [128]:
df_boy_groups

,Name,Short,Korean Name,Debut,Company,Members,Orig. Memb.,Fanclub Name,Active
0,100%,NaN,백퍼센트,2012-09-18,TOP Media,4,7,Perfection,Yes
1,14U,NaN,원포유,2017-04-17,BG,14,14,NaN,Yes
2,1the9,NaN,원더나인,2019-02-09,MBK,9,9,NaN,Yes
3,24K,NaN,투포케이,2012-09-06,Choeun,8,6,24U,Yes
4,2AM,NaN,투에이엠,2008-06-21,"JYP, Big Hit",4,4,I Am,No
...,...,...,...,...,...,...,...,...,...
142,VROMANCE,NaN,브로맨스,2016-06-12,RBW,4,4,NaN,Yes
143,Wanna One,NaN,워너원,2017-08-07,YMC,11,11,Wannable,No
144,We In The Zone,NaN,위인더존,2019-05-27,Choon,5,5,NaN,Yes
145,WINNER,NaN,위너,2014-08-12,YG,4,5,Ace,Yes


In [129]:
df_girl_groups

,Name,Short,Korean Name,Debut,Company,Members,Orig. Memb.,Fanclub Name,Active
0,(G)I-DLE,NaN,(여자)아이들,2018-05-02,Cube,6,6,NaN,Yes
1,15&,NaN,피프틴앤드,2012-10-05,JYP,2,2,NaN,No
2,2EYES,NaN,투아이즈,2013-07-20,SidusHQ,4,5,NaN,No
3,2NE1,NaN,투애니원,2009-05-06,YG,4,4,Blackjack,No
4,3YE,NaN,써드아이,2019-05-21,GH,3,3,NaN,Yes
...,...,...,...,...,...,...,...,...,...
147,We Girls,NaN,위걸스,2018-08-31,Afternoon,8,8,NaN,Yes
148,Weki Meki,WeMe,위키미키,2017-08-08,Fantagio,8,8,NaN,Yes
149,WJSN,Cosmic Girls,우주소녀,2016-02-25,Starship,13,12,Ujung,Yes
150,Wonder Girls,WG,원더걸스,2007-02-13,JYP,4,5,Wonderful,No


**Final dataframe for the kpop artists**

In [130]:
df_allgroups = pd.concat([df_girl_groups, df_boy_groups])
df_allgroups.drop(['Short'], axis=1, inplace=True)
df = pd.merge(df_idols, df_allgroups, how='left', left_on='Group', right_on='Name')
df.drop(['Name', 'Korean Name_y'], axis=1, inplace=True)
df

,Stage Name,Full Name,Korean Name_x,K. Stage Name,Date of Birth,Group,Country,Birthplace,Other Group,Gender,Debut,Company,Members,Orig. Memb.,Fanclub Name,Active
0,A.M,Seong Hyunwoo,성현우,에이엠,1996-12-31,Limitless,South Korea,NaN,NaN,M,2019-07-09,ONO,4.0,4.0,NaN,Yes
1,Ace,Jang Wooyoung,장우영,에이스,1992-08-28,VAV,South Korea,NaN,NaN,M,2015-10-31,A team,7.0,6.0,NaN,Yes
2,Aeji,Kwon Aeji,권애지,애지,1999-10-25,Hashtag,South Korea,Daegu,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN
3,AhIn,Lee Ahin,이아인,아인,1999-09-27,MOMOLAND,South Korea,Wonju,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN
4,Ahra,Go Ahra,고아라,아라,2001-02-21,Favorite,South Korea,Yeosu,NaN,F,2017-07-05,Astory,6.0,6.0,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,Ziu,Park Heejun,박희준,지우,1997-06-16,VAV,South Korea,NaN,NaN,M,2015-10-31,A team,7.0,6.0,NaN,Yes
1306,ZN,Bae Jinye,배진예,지엔,1994-06-09,LABOUM,South Korea,Bucheon,UNI.T,F,NaN,NaN,NaN,NaN,NaN,NaN
1307,Zoa,Cho Hyewon,조혜원,조아,2005-05-31,Weeekly,South Korea,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN
1308,Zuho,Bae Juho,백주호,주호,1996-07-04,SF9,South Korea,NaN,NaN,M,2016-10-05,FNC,9.0,9.0,Fantasy,Yes


**Example: Group "Mamamoo" members data**

In [131]:
df[df['Group'].isin(['(G)I-DLE'])]

,Stage Name,Full Name,Korean Name_x,K. Stage Name,Date of Birth,Group,Country,Birthplace,Other Group,Gender,Debut,Company,Members,Orig. Memb.,Fanclub Name,Active
774,Minnie,Nicha Yontararak,니차 욘타라락,민니,1997-10-23,(G)I-DLE,Thailand,NaN,NaN,F,2018-05-02,Cube,6.0,6.0,NaN,Yes
794,Miyeon,Jo Miyeon,조미연,미연,1997-01-31,(G)I-DLE,South Korea,NaN,NaN,F,2018-05-02,Cube,6.0,6.0,NaN,Yes
969,Shuhua,Yeh Shuhua,예슈화,슈화,2000-01-06,(G)I-DLE,Taiwan,NaN,NaN,F,2018-05-02,Cube,6.0,6.0,NaN,Yes
1005,Soojin,Seo Soojin,서수진,수진,1998-03-09,(G)I-DLE,South Korea,NaN,NaN,F,2018-05-02,Cube,6.0,6.0,NaN,Yes
1015,Soyeon,Jeon Soyeon,전소연,소연,1998-08-26,(G)I-DLE,South Korea,NaN,NaN,F,2018-05-02,Cube,6.0,6.0,NaN,Yes
1291,Yuqi,Song Yuqi,송우기,우기,1999-09-23,(G)I-DLE,China,Beijing,NaN,F,2018-05-02,Cube,6.0,6.0,NaN,Yes


<h1>分水嶺<h1>

<h2>Nodes, Edges<h2>

In [22]:
rank = scrapeMelon.getList("LIVE").decode()
rank

'{"1": {"name": "사건의 지평선", "artists": "윤하 (YOUNHA)", "ranking": 1, "songId": "34819473", "albumId": "10903868"}, "2": {"name": "ANTIFRAGILE", "artists": "LE SSERAFIM (르세라핌)", "ranking": 2, "songId": "35729104", "albumId": "11078496"}, "3": {"name": "Nxde", "artists": "(여자)아이들", "ranking": 3, "songId": "35730562", "albumId": "11078852"}, "4": {"name": "Hype boy", "artists": "NewJeans", "ranking": 4, "songId": "35454426", "albumId": "11011565"}, "5": {"name": "After LIKE", "artists": "IVE (아이브)", "ranking": 5, "songId": "35546497", "albumId": "11034298"}, "6": {"name": "새삥 (Prod. ZICO) (Feat. 호미들)", "artists": "지코 (ZICO)", "ranking": 6, "songId": "35595136", "albumId": "11045985"}, "7": {"name": "Attention", "artists": "NewJeans", "ranking": 7, "songId": "35454425", "albumId": "11011565"}, "8": {"name": "Rush Hour (Feat. j-hope of BTS)", "artists": "Crush", "ranking": 8, "songId": "35665282", "albumId": "11062364"}, "9": {"name": "Monologue", "artists": "테이", "ranking": 9, "songId": "356

In [23]:
rank_dict = ast.literal_eval(rank) # Make this "string" -> "dict"

In [24]:
rank_en = dict([(key, ts.google(str(value), from_language='ko', to_language='en')) for key, value in rank_dict.items()])
rank_en['44'] = rank_en['44'] + '}'
rank_en['88'] = rank_en['88'] + '}'
rank_en

{'1': "{'Name': 'Horizon of the Incident', 'Artists': 'YOUNHA', 'Ranking': 1, 'Songid': '34819473', 'Albumid': '10903868'}",
 '2': "{'Name': 'AntiFragile', 'Artists': 'Le Sserafim', 'Ranking': 2, 'Songid': '35729104', 'Albumid': '11078496'}",
 '3': "{'Name': 'NXDE', 'Artists': '(Women)', 'Ranking': 3, 'Songid': '35730562', 'Albumid': '11078852'}",
 '4': "{'Name': 'Hype Boy', 'Artists': 'Newjeans', 'Ranking': 4, 'Songid': '35454426', 'Albumid': '11011565'}",
 '5': "{'Name': 'After Like', 'Artists': 'Ive (Ive)', 'Ranking': 5, 'Songid': '35546497', 'Albumid': '11034298'}",
 '6': "{'Name': 'Prod. Zico (Feat. Homi)', 'Artists':' Zico ',' Ranking ': 6,' Songid ':' 35595136 ',' Albumid ':' 11045985 '}",
 '7': "{'Name': 'Attension', 'Artists': 'Newjeans', 'Ranking': 7, 'Songid': '35454425', 'Albumid': '11011565'}",
 '8': "{'Name': 'Rush Hour (Feat. J-Hope of BTS)', 'Artists': 'Crush', 'Ranking': 8, 'Songid': '35665282', 'Albumid': '11062364'}",
 '9': "{'Name': 'Monologue', 'Artists': 'Tei', 'R

In [78]:
songs_df = []
for key, j in rank_en.items():
    try:
        res = re.sub('\(Women\)', '(G)I-DLE', j)
        res = re.sub('Ive \(Ive\)', 'Ive', res)
        res = re.sub('Blackpink', 'BLACKPINK', res)
        res = re.sub(' PSY ', 'PSY', res)
        res = re.sub('Itzy', 'ITZY', res)
        res = re.sub('Dreamers [Music from the FIFA WORLD CUP qatar 2022 Official Soundtrack]', 
                     'Dreamers [Music from the FIFA WORLD CUP qatar 2022 Official Soundtrack]\'', res)
        res = re.sub(' Jungkook, BTS ', 'Jungkook', res)
        res = re.sub('It\'s Love', 'It%27s Love', res)
        result = ast.literal_eval(res)
        result = {k.lower(): v for k, v in result.items()}
        songs_df.append(pd.DataFrame([result]))
    except:
        pass

song_df = pd.concat(songs_df)
#song_df = song_df.assign(**{'songID': song_df['songid'].fillna(song_df['songs'])})
#song_df = song_df.assign(**{'albumID': song_df['albumid'].fillna(song_df['albums'])})
#song_df.drop(['songid', 'albumid', 'songs', 'albums', ' ranking ', ' songid ', ' albumid '], axis=1, inplace=True)
song_df.drop([' ranking ', ' songid ', ' albumid '], axis=1, inplace=True)
song_df.dropna(axis=0, inplace=True)
song_df.set_index(np.arange(1,len(song_df)+1), inplace=True)
song_df.replace(' ', '_', regex=True, inplace=True)
song_df

,name,artists,ranking,songid,albumid
1,Horizon_of_the_Incident,YOUNHA,1.0,34819473,10903868
2,AntiFragile,Le_Sserafim,2.0,35729104,11078496
3,NXDE,(G)I-DLE,3.0,35730562,11078852
4,Hype_Boy,Newjeans,4.0,35454426,11011565
5,After_Like,Ive,5.0,35546497,11034298
...,...,...,...,...,...
79,The_Astronaut,Jin,96.0,35779025,11089457
80,You_are_so_great,Lim_Young_-woong,97.0,35008526,10923444
81,Ghost_Town,Benson_Boone,98.0,34098217,10743130
82,All_I_want_for_Christmas_is_you,Mariah_Carey,99.0,14776,829


In [85]:
frames = [song_df[song_df['artists'].isin(df['Group'])], song_df[song_df['artists'].isin(df['Stage Name'])]]
df_song = pd.concat(frames, axis=0)
df_song.sort_index(inplace=True)
df_song

,name,artists,ranking,songid,albumid
3,NXDE,(G)I-DLE,3.0,35730562,11078852
10,Shut_Down,BLACKPINK,11.0,35640077,11033394
11,PINK_VENOM,BLACKPINK,12.0,35542908,11033394
40,Sneakers,ITZY,49.0,35413033,11004992
44,Dynamite,BTS,53.0,32872978,10479150
50,Catch_my_hand,IU,60.0,3414749,1286252
57,Strawberry_Moon,IU,67.0,34101563,10743453
58,Butter,BTS,68.0,33507137,10612483
61,Drama,IU,71.0,34481680,10827816
64,Spring_Day,BTS,76.0,30244931,10037969


<h1>已蒐集完“node”資料<h1>

In [23]:
from pathlib import Path
mother_dir = Path("Rank_eachArtist")

In [78]:
#The groups we interested in (Which are in TOP 100 in Melon)
group_name = []
for name in list(mother_dir.glob("*.csv")):
    name = str(name)
    name = name.replace("Rank_eachArtist/", "")
    name = name.replace(".csv", "")
    group_name.append(name)
print(group_name)

['Aespa', 'Girls-Generation', 'Younha', 'BTS', 'ITZY', '(G)-IDLE', 'Red_Velvet', 'IU', 'Twice', 'Ive', 'Le_Sserafim', 'BLACKPINK', 'Bigbang', 'NewJeans', 'NMIXX']


In [84]:
#Each groups' dataframe
Le_Sserafim_df = Le_Sserafim_df = pd.read_csv("Rank_eachArtist/" + "Le_Sserafim" + ".csv")
if 'Unnamed: 0' in Le_Sserafim_df:
    Le_Sserafim_df.drop(['Unnamed: 0'], axis=1, inplace=True)
Le_Sserafim_df['group'] = 'Le_Sserafim'
Le_Sserafim_df['group_wiki'] = 'le-sserafim'
    
Girls_Generation_df = pd.read_csv("Rank_eachArtist/" + "Girls-Generation" + ".csv")
if 'Unnamed: 0' in Girls_Generation_df:
    Girls_Generation_df.drop(['Unnamed: 0'], axis=1, inplace=True)
Girls_Generation_df['group'] = 'Girls-Generation' 
Girls_Generation_df['group_wiki'] = 'girls-generation'
    
BTS_df = pd.read_csv("Rank_eachArtist/" + "BTS" + ".csv")
if 'Unnamed: 0' in BTS_df:
    BTS_df.drop(['Unnamed: 0'], axis=1, inplace=True)
BTS_df['group'] = 'BTS'
BTS_df['group_wiki'] = 'bts'
    
ITZY_df = pd.read_csv("Rank_eachArtist/" + "ITZY" + ".csv")
if 'Unnamed: 0' in ITZY_df:
    ITZY_df.drop(['Unnamed: 0'], axis=1, inplace=True)
ITZY_df['group'] = 'ITZY'
ITZY_df['group_wiki'] = 'itzy'
    
G_IDLE_df = pd.read_csv("Rank_eachArtist/" + "(G)-IDLE" + ".csv")
if 'Unnamed: 0' in G_IDLE_df:
    G_IDLE_df.drop(['Unnamed: 0'], axis=1, inplace=True)
G_IDLE_df['group'] = '(G)-IDLE'
G_IDLE_df['group_wiki'] = 'g-i-dle'
    
Red_Velvet_df = pd.read_csv("Rank_eachArtist/" + "Red_Velvet" + ".csv")
if 'Unnamed: 0' in Red_Velvet_df:
    Red_Velvet_df.drop(['Unnamed: 0'], axis=1, inplace=True)
Red_Velvet_df['group_wiki'] = 'Red_Velvet'
Red_Velvet_df['group_wiki'] = 'redvelvet'
    
IU_df = pd.read_csv("Rank_eachArtist/" + "IU" + ".csv")
if 'Unnamed: 0' in IU_df:
    IU_df.drop(['Unnamed: 0'], axis=1, inplace=True)
IU_df['group'] = 'IU'
IU_df['group_wiki'] = 'i-u'
    
Twice_df = pd.read_csv("Rank_eachArtist/" + "Twice" + ".csv")
if 'Unnamed: 0' in Twice_df:
    Twice_df.drop(['Unnamed: 0'], axis=1, inplace=True)
Twice_df['group'] = 'Twice'
Twice_df['group_wiki'] = 'twice'
    
Aespa_df = pd.read_csv("Rank_eachArtist/" + "Aespa" + ".csv")
if 'Unnamed: 0' in Aespa_df:
    Aespa_df.drop(['Unnamed: 0'], axis=1, inplace=True)
Aespa_df['group'] = 'Aespa'
Aespa_df['group_wiki'] = 'aespa'
    
Ive_df = pd.read_csv("Rank_eachArtist/" + "Ive" + ".csv")
if 'Unnamed: 0' in Ive_df:
    Ive_df.drop(['Unnamed: 0'], axis=1, inplace=True)
Ive_df['group'] = 'Ive'
Ive_df['group_wiki'] = 'ive'
    
BLACKPINK_df = pd.read_csv("Rank_eachArtist/" + "BLACKPINK" + ".csv")
if 'Unnamed: 0' in BLACKPINK_df:
    BLACKPINK_df.drop(['Unnamed: 0'], axis=1, inplace=True)
BLACKPINK_df['group'] = 'BLACKPINK'
BLACKPINK_df['group_wiki'] = 'black-pink'
    
Bigbang_df = pd.read_csv("Rank_eachArtist/" + "Bigbang" + ".csv")
if 'Unnamed: 0' in Bigbang_df:
    Bigbang_df.drop(['Unnamed: 0'], axis=1, inplace=True)
Bigbang_df['group'] = 'Bigbang'
Bigbang_df['group_wiki'] = 'bigbangkpop'
    
NewJeans_df = pd.read_csv("Rank_eachArtist/" + "NewJeans" + ".csv")
if 'Unnamed: 0' in NewJeans_df:
    NewJeans_df.drop(['Unnamed: 0'], axis=1, inplace=True)
NewJeans_df['group'] = 'NewJeans'
NewJeans_df['group_wiki'] = 'newjeans'
    
NMIXX_df = pd.read_csv("Rank_eachArtist/" + "NMIXX" + ".csv")
if 'Unnamed: 0' in NMIXX_df:
    NMIXX_df.drop(['Unnamed: 0'], axis=1, inplace=True)
NMIXX_df['group'] = 'NMIXX'
NMIXX_df['group_wiki'] = 'nmixx'

In [85]:
#Combine all the dataframes
frames = [Le_Sserafim_df, Girls_Generation_df, BTS_df, ITZY_df, 
          G_IDLE_df, Red_Velvet_df, IU_df, Twice_df, Aespa_df,
          Ive_df, BLACKPINK_df, Bigbang_df, NewJeans_df, NMIXX_df]

all_df = pd.concat(frames)
all_df.fillna(0)
#all_df.song.combine_first(all_df.Song)
#all_df.resource.combine_first(all_df.Resourse)
#all_df.drop(['Song', 'Resourse'], axis=1, inplace=True)
all_df.reset_index(inplace=True)
all_df.drop(['index'], axis=1, inplace=True)
all_df

,NO,song,resource,Web,group,group_wiki
0,1,ANTIFRAGILE_(song),Wiki_f,https://le-sserafim.fandom.com/wiki/ANTIFRAGIL...,Le_Sserafim,le-sserafim
1,2,Fearless_(Le_Sserafim_song),Wiki,NaN,Le_Sserafim,le-sserafim
2,3,impurities,NaN,NaN,Le_Sserafim,le-sserafim
3,4,Good Parts (when the quality is bad but I am),NaN,NaN,Le_Sserafim,le-sserafim
4,5,no celestial,NaN,NaN,Le_Sserafim,le-sserafim
...,...,...,...,...,...,...
203,3,Funky_Glitter_Christmas,Wiki_f,https://kpop.fandom.com/wiki/Funky_Glitter_Chr...,NMIXX,nmixx
204,4,COOL_(Your_rainbow),Wiki_f,https://nmixx.fandom.com/wiki/COOL_(Your_rainbow),NMIXX,nmixx
205,5,Kiss,Wiki_f,https://nmixx.fandom.com/wiki/Kiss,NMIXX,nmixx
206,6,TANK,Wiki_f,https://nmixx.fandom.com/wiki/TANK,NMIXX,nmixx


<h2>Wikipage Scraping<h2>

**Scrapt our data from the website**

In [102]:
http = urllib3.PoolManager()

def wiki_scr(text):
   baseurl = "https://en.wikipedia.org/w/api.php?"
   action = "action=query"
   title = "titles=" + text
   content = "prop=revisions&rvprop=content"
   dataformat = "format=json"
   
   query = "%s%s&%s&%s&%s" % (baseurl, action, content, title, dataformat)
   wiki_bf(text, query)

def wiki_kpop_scr(text):
   baseurl = "https://kpop.fandom.com/api.php?"
   action = "action=query"
   title = "titles=" + text
   content = "prop=revisions&rvprop=content"
   dataformat = "format=json"

   query = "%s%s&%s&%s&%s" % (baseurl, action, content, title, dataformat)
   wiki_bf(text, query)
   

def wiki_scr_kpopEach(gname, text):
   baseurl = "https://" + gname + ".fandom.com/api.php?"
   action = "action=query"
   title = "titles=" + text
   content = "prop=revisions&rvprop=content"
   dataformat = "format=json"

   query = "%s%s&%s&%s&%s" % (baseurl, action, content, title, dataformat)
   wiki_bf(text, query)
   

def wiki_bf(text, query):
   wikiresource = http.request('GET', query)
   wikijson = json.loads(wikiresource.data.decode('utf-8'))
   
   if re.search(r'/', text):
      text = text.replace("/", "\\")
    
   with open('/Users/luchengliang/Social_Graph_and_Networks/Final_Project/jsonWebpage/' + text + '.txt', 'w') as f:
      f.write(str(wikijson))

In [101]:
for i in range(len(all_df)):
   x = all_df.iloc[i, :]
   
   #Scrape different kind of wiki pages
   #First is the normal wiki
   if x['resource'] == 'Wiki':
      q = wiki_scr(x['song'])

   if x['resource'] == 'Wiki_f':
      #Second one is from Kpop fandom page, not their own page
      if '//kpop.fandom' in str(x['Web']):
         q = wiki_kpop_scr(x['song'])
      elif type(x['group_wiki']) != float:
         #Third one is from each fandom wiki page
         q = wiki_scr_kpopEach(x['group_wiki'], x['song'])
      else:
         continue

In [104]:
path = "/Users/luchengliang/Social_Graph_and_Networks/Final_Project/jsonWebpage"

dir_list = os.listdir(path)
    
print("Files: ") 
# print the list
print(dir_list)

Files: 
['Typa_Girl.txt', 'Change.txt', 'Tell_Me_Your_Wish_(Genie)_(song).txt', 'Never_Stop_Me.txt', 'Black_Swan.txt', 'Happiness.txt', 'Butter_(song).txt', 'Bang_Bang_Bang.txt', 'Dance_The_Night_Away.txt', 'Hurt.txt', 'SNEAKERS.txt', 'Lovesick_Girls.txt', 'Talk_That_Talk.txt', 'The_Truth_Untold.txt', 'Cookie.txt', 'Hype_Boy.txt', 'Oh!_(song).txt', 'Blue.txt', 'I_Need_U_(BTS_song).txt', 'In_the_morning.txt', 'Merry_%26_Happy.txt', 'After_LIKE.txt', '.DS_Store', 'Christmas_Carol_Medley.txt', 'Butterfly.txt', 'Wish_Tree.txt', 'Magic_Shop.txt', 'Not_Shy.txt', 'Like.txt', 'Twenty.txt', 'ICY.txt', 'Sour_Grapes.txt', 'Illusion.txt', 'Boy_with_Luv.txt', 'Answer:_Love_Myself.txt', 'Fantastic_Baby.txt', 'Yes_Or_Yes.txt', 'Nxde.txt', 'Anpanman.txt', 'Ice_Cream_Cake_(song).txt', 'Take_It.txt', 'Stay.txt', 'LOCO.txt', 'Dreams_Come_True.txt', 'Left_and_Right_(Feat._Jung_Kook_of_BTS).txt', 'Palette.txt', 'HWAA.txt', 'Epiphany.txt', 'Don%27t_Know_What_To_Do.txt', 'Heart_Shaker.txt', 'Luv_U.txt', 'Psy

<h1>Building Network<h1>

In [87]:
#Find out the writer, producer, song writer, distributors
with open('./jsonWebpage/ANTIFRAGILE_(song).txt') as f:
    Anti = f.readlines()
    connections = re.findall(r'\|writer(.*?)\|', str(Anti))
    connections = str(connections).partition('=')
    connections = connections[2]
    connections = connections.replace("\']", "").lower()
    connections = connections.split(", ")
connections

['supreme boi',
 'ronnie icon',
 'shintaro yasuda',
 'paulina "pau" cerrilla',
 'kyler niko']

In [125]:
#Find out the writer, producer, song writer, distributors
with open('./jsonWebpage/Fearless_(Le_Sserafim_song).txt') as f:
    Anti = f.readlines()
    connections = re.findall(r'\| writer(.*?)\|', str(Anti))
    connections = str(connections).partition('=')
    connections = connections[2]
    connections = connections.replace("\\n*", ",")
    connections = connections.split("\,")
    print(connections)
    print(type(connections))
    connections = ''.join(char for char in connections if char.isalnum() or char.isspace())
    #connections = connections.split("n")
connections

['\\\\', ' Score (13)\\\\', ' Megatone (13)\\\\', ' Supreme Boi\\\\', ' Blvsh\\\\', ' Jaro\\\\', ' Nikolay Mohr\\\\', " [[Bang Si-hyuk']"]
<class 'list'>


''

In [ ]:
#Find out the writer, producer, song writer, distributors
with open('./jsonWebpage/ANTIFRAGILE_(song).txt') as f:
    Anti = f.readlines()
    connections = re.findall(r'\|writer=[^\W_]*\|', str(Anti))
    for i in range(len(connections)):
        connections[i] = connections[i].partition('|')[0]
        connections[i] = connections[i].replace(" ", "_")
    print(set(connections))


path_dc = "/Users/luchengliang/Social_Graph_and_Networks/DC"
path_marvel = "/Users/luchengliang/Social_Graph_and_Networks/Marvel"

dir_list_dc = os.listdir(path_dc)
dir_list_marvel = os.listdir(path_marvel)

neighbors = []
#print(connections)
for j in range(len(connections)):
    for p in range(len(dir_list_dc)):
        dir_list_dc[p] = dir_list_dc[p].replace(".txt", "")
        neighbors += re.findall(dir_list_dc[p], connections[j])
        
    for u in range(len(dir_list_marvel)):
        dir_list_marvel[u] = dir_list_marvel[u].replace(".txt", "")
        neighbors += re.findall(dir_list_marvel[u], connections[j])

#print(neighbors)

<h2>Lyrics<h2>

In [ ]:
content = scrapeMelon.getLyric(35729104)
print(content)
print(type(content))

Anti ti ti ti fragile fragile
Anti ti ti ti fragile
Anti ti ti ti fragile fragile
Antifragile antifragile
가시밭길 위로 riding
you made me boost up
거짓으로 가득 찬 party
가렵지도 않아
내 뒤에 말들이 많아
나도 첨 듣는 내 rival
모두 기도해 내 falling
그 손 위로 I’mma jump in
Yes gimme that
걸어봐 위엄 like a lion
눈빛엔 거대한 desire
더 부어 gasoline on fire
불길 속에 다시 날아 rising
잊지 마 내가 두고 온 toe shoes
무슨 말이 더 필요해
무시 마 내가 걸어온 커리어
I go to ride till I die die
더 높이 가줄게
내가 바랐던 세계 젤 위에
떨어져도 돼
I’m antifragile antifragile
난 지금 on my way 갖다버려 줘
너의 fairy tale
Now you know my name
I’m antifragile antifragile
Anti ti ti ti fragile fragile
Anti ti ti ti fragile
Anti ti ti ti fragile fragile
Antifragile antifragile
“Lovey lovey lovey
dovey dovey dovey”
멋대로 정하네 나란 애에 대해
I don't know what to say
I can't feel it
뜨거운 관심은 환영
귀여운 질투는 go ahead
줄 달린 인형은 no thanks
내 미랠 쓸 나의 노래
Yes gimme that
걸어봐 위엄 like a lion
눈빛엔 거대한 desire
더 부어 gasoline on fire
불길 속에 다시 날아 rising
잊지 마 내가 두고 온 toe shoes
무슨 말이 더 필요해
무시 마 내가 걸어온 커리어
I go to ride till I die die
더 높이 가줄게
내가 바랐던 세계 젤 위에


In [ ]:
help(ts.google)

Help on method google_api in module translators.servers:

google_api(query_text: str, from_language: str = 'auto', to_language: str = 'en', **kwargs) -> Union[str, list] method of translators.servers.GoogleV2 instance
    https://translate.google.com, https://translate.google.cn.
    :param query_text: str, must.
    :param from_language: str, default 'auto'.
    :param to_language: str, default 'en'.
    :param **kwargs:
            :param reset_host_url: str, default None. eg: 'https://translate.google.fr'
            :param if_use_cn_host: boolean, default None. affected by `reset_host_url`.
            :param if_ignore_limit_of_length: boolean, default False.
            :param is_detail_result: boolean, default False.
            :param timeout: float, default None.
            :param proxies: dict, default None.
            :param sleep_seconds: float, default `random.random()`.
    :return: str or list



In [ ]:
content_trans = ts.google(content, from_language='auto', to_language='en')
print(content_trans)

Anti Ti Ti Ti fragile fragile
Anti Ti Ti Ti Fragile
Anti Ti Ti Ti fragile fragile
AntiFragile Antifragile
Riding on the thorny field road
You Made Me Boost Up
PARTY full of lies
Not itchy
There are a lot of horses behind me
My RIVAL
All prayer
I ’mma jump in that hand
Yes gimme that
Walk Dignity like a Lion
A huge desire for eyes
Pour more Gasoline on Fire
RISING flew again in the flames
Don't forget TOE Shoes I left
I need something more
The career I've walked
I go to ride till I die Die
I'll get more height
On the world gel I wanted
You can fall
I ’m Antifragile Antifragile
I'm on my way now
Your Fairy Tale
Now you know my name
I ’m Antifragile Antifragile
Anti Ti Ti Ti fragile fragile
Anti Ti Ti Ti Fragile
Anti Ti Ti Ti fragile fragile
AntiFragile Antifragile
“LOVEY LOVEY LOVEY
DOVEY DOVEY DOVEY ”
I decided to decide what it is.
I don't know what to say
I Can't Feel It
Hot interest is welcome
Cute jealousy is Go ahead
The rowing doll is No Thanks
My song to write my mirror
Yes gimme